# Splitting solvers for Biot's equations

## Recap the block structure of the problem

Biot's equations couple elasticity and fluid flow equations. There is various formulations for the coupled problem. In PorePy Biot's equations are discretized by the finite volume method applied to the two-field formulation. The corresponding unknown fields are the displacement $\mathbf{u}$ and the fluid pressure $p$.

The two-field formulation of Biot's equations reads: Find $\mathbf{u}$ and $p$ satisfying in $\Omega$ 
\begin{align}
-\mathbf{\nabla} \cdot \left( 2\mu \mathbf{\varepsilon}(\mathbf{u}) + \lambda \mathrm{Tr}\,\mathbf{\varepsilon}(\mathbf{u})\, \mathbf{I} - \alpha p \mathbf{I}\right) &= \mathbf{b} \\
s_0\partial_t p + \alpha \partial_t \mathrm{Tr}\,\mathbf{\varepsilon}(\mathbf{u}) + \mathbf{\nabla} \cdot (-\mathbf{K} \mathbf{\nabla} p) &= \psi,
\end{align}
at initial time $t=0$
\begin{alignat}{2}
\mathbf{u} &= \mathbf{u}_0  \\
p &= p_0,
\end{alignat}
and on the boundary $\partial \Omega$
\begin{alignat}{2}
\mathbf{u} &= \mathbf{u}_\mathrm{D} &\quad&\text{on }\Gamma_\mathrm{u,D}\\
\left(2\mu \mathbf{\varepsilon}(\mathbf{u}) + \lambda \mathrm{Tr}\,\mathbf{\varepsilon}(\mathbf{u})\, \mathbf{I} - \alpha p \mathbf{I}\right)\cdot \mathbf{n} &= \mathbf{\sigma}_\mathrm{N} &\quad&\text{on }\Gamma_\mathrm{u,N}\\
\mathbf{p} &= \mathbf{p}_\mathrm{D} &\quad&\text{on }\Gamma_\mathrm{p,D}\\
\left(-\mathbf{K}\mathbf{\nabla}p \right) \cdot \mathbf{n} &= \mathbf{\sigma}_\mathrm{N} &\quad&\text{on }\Gamma_\mathrm{p,N}
\end{alignat}
for two partitions $\bar{\Gamma}_\mathrm{u,D} \cup \bar{\Gamma}_\mathrm{u,N}=\bar{\Gamma}_\mathrm{p,D} \cup \bar{\Gamma}_\mathrm{p,N}=\partial\Omega$.

After finite volume discretization, using MPSA for elasticity and MPFA for flow, with cell-centered displacements and pressures as primary degrees of freedom, we obtain al algebraic problem with natural block structure
\begin{align}
\begin{bmatrix} \mathbf{A} & -\mathbf{B}^\top \\ \mathbf{B} & \mathbf{C} \end{bmatrix} 
\begin{bmatrix} \mathbf{u} \\ \mathbf{p} \end{bmatrix} 
= \begin{bmatrix} \mathbf{f} \\ \mathbf{g} \end{bmatrix}
\end{align}

## Setting up a footing problem in PorePy

We consider a unit square in 2D. We fix its displacement and apply no flow condition on the bottom boundary. In addition, we apply a force to the top boundary, but only half of it at the center of the boundary. Here, also no flow boundary conditions are applied. The remaining boundary conditions are zero stress and zero pressure boundary conditions.


We will require some standard packages.

In [1]:
import numpy as np
import porepy as pp
import scipy.sparse as sps

In /home/jakub/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/jakub/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/jakub/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/jakub/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/jakub/anaconda3/lib/python3.7/site-packages/matplotlib/

Define problem parameters. In particular, define the material parameters $\mu = \text{lame_mu}$, $\lambda = \texttt{lame_lambda}$, $K = \texttt{kappa}$ (actually a tensor), $s_0 = \texttt{s0}$, and $\alpha = \texttt{alpha}$. In addition, we can choose discretization parameters for the mesh resolution and time step size.

In [2]:
model_params = {
        # DO NOT CHANGE THESE:
        "file_name" : "poroelasticity",   # exporter
        "folder_name" : "out",  # expoter
        "use_ad" : True,
        # FEEL FREE TO CHANGE ANY OF THOSE:
        "lame_mu": 1e8,
        "lame_lambda": 1e7,
        "force": -1e6,
        "kappa": 1e-12,
        "s0": 0.5e-7, # * 0.07, 0.17, *0.19, *2
        "alpha": 1.,
        "mesh_resolution": 80,
        "time_step": 0.25,
        "fixed_stress_L": 1, #0.9,
        "aa_depth": 0, #5
        }

Now we simply define the footing problem (external module) for given model parameters, including all boundary conditions, source terms etc.; we also run an preparation routine defining the grid, assign equations and initialize algebraic data structures.

In [3]:
from footingproblem_model import FootingProblem
model = FootingProblem(model_params)
model.prepare_simulation()

Now we need to solve the problem. We consider the three approaches: monolithic, straight-forward Block-Gauss Seidel, and stabilized Block-Gauss Seidel.

## Monolithic solution
The problem is linear, and we use direct solvers. Nevertheless, models in PorePy are currently implemented as general nonlinear models, providing Jacobians and residuals when assembling. Hence, we use the approach Newton's method here, to solve the problem. It will converge in 1 iteration.

To recall, in incremental form, iteration $i$ reads: Given $\mathbf{u}^{i-1}$ and $p^{i-1}$, find $\mathbf{u}^i := \mathbf{u}^{i-1} + \Delta \mathbf{u}^i$ and $p^i := p^{i-1} + \Delta p^{i}$ with the increments satisfying

\begin{align}
\begin{bmatrix} \mathbf{A} & -\mathbf{B}^\top \\ \mathbf{B} & \mathbf{C} \end{bmatrix} 
\begin{bmatrix} \Delta \mathbf{u}^i \\ \Delta \mathbf{p}^i \end{bmatrix} 
= \begin{bmatrix} \mathbf{f} \\ \mathbf{g} \end{bmatrix} - \begin{bmatrix} \mathbf{A} & -\mathbf{B}^\top \\ \mathbf{B} & \mathbf{C} \end{bmatrix} 
\begin{bmatrix} \mathbf{u}^{i-1} \\ \mathbf{p}^{i-1} \end{bmatrix} 
\end{align}

So on the right hand side, we have the standard residual, while the left hand side we use the exact Jacobian.

Let's setup the algebraic structures required for increments and residuals.

In [4]:
num_dofs = model.num_dofs()
increment = np.zeros(num_dofs)
residual = np.zeros(num_dofs)

Next, we introduce some auxiliary variables to be used in the solver strategy and construct the data structures, to be used in the later solution process.

In [5]:
first_call = True
time_step = 0

The time loop and iterative solver loop.

In [6]:
while model.time < model.end_time:
    # Time management
    time_step += 1
    model.time += model.time_step

    print(f"+=====================================+\nTime step {time_step}:") 

    # Newton loop
    model.before_newton_loop()
    while model.convergence_status == False:

        # Empty method, but include for completeness
        model.before_newton_iteration()

        # Set up the Jacobian and factorize it - only in the first iteration.
        if first_call:
            mat, _ = model.assemble_linear_system()
            mat_splu = sps.linalg.splu(mat)
            first_call = False
            
        # Assemble the residual
        _, residual = model.assemble_linear_system()
        
        # Compute the increment by performing one Newton step
        increment = mat_splu.solve(residual)
        
        # Distribute the increment
        model.distribute_solution(increment)

        # Manage the iteration (accumulate to determine the solution),
        # and monitor convergence. Stop, if residual is below tolerance
        is_converged = model.after_newton_iteration(increment, residual, tol=1e-6)
        
        # Export solution
        if is_converged:
            model.after_newton_convergence()
            model.export(time_step)

    print(f"+=====================================+\n")

+=====================================+
Time step 1:


/home/jakub/anaconda3/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


Iter. 1 | Residual 79057.50171297793
Iter. 2 | Residual 1.0368555704874683e-07
+=====================================+

+=====================================+
Time step 2:
Iter. 1 | Residual 1.714936351511903e-07
+=====================================+

+=====================================+
Time step 3:
Iter. 1 | Residual 1.5859813165787892e-07
+=====================================+

+=====================================+
Time step 4:
Iter. 1 | Residual 1.4690874753113663e-07
+=====================================+



The solution (for the initial parameters) should look something like this (you see the geometry warped by the displacement and the colouring corresponds to the pressure).

![title](out/warped-pressure-cut2.png)

## Fixed strain - the naive Block-Gauss Seidel approach
We investigate block-structured solver strategies for Biot's equations and start with the straigh-forward Block-Gauss Seidel approach. (Recall: this solver is only conditionally stable.) The idea of the solver is to simply decouple the elasticity and fluid flow equations and solve the successively until (possible) convergence.

To recall, in incremental form, iteration $i$ reads: Given $\mathbf{u}^{i-1}$ and $p^{i-1}$, find $\mathbf{u}^i := \mathbf{u}^{i-1} + \Delta \mathbf{u}^i$ and $p^i := p^{i-1} + \Delta p^{i}$ with the increments satisfying

\begin{align}
\begin{bmatrix} \mathbf{A} & -\mathbf{B}^\top \\ \mathbf{0} & \mathbf{C} \end{bmatrix} 
\begin{bmatrix} \Delta \mathbf{u}^i \\ \Delta \mathbf{p}^i \end{bmatrix} 
= \begin{bmatrix} \mathbf{f} \\ \mathbf{g} \end{bmatrix} - \begin{bmatrix} \mathbf{A} & -\mathbf{B}^\top \\ \mathbf{B} & \mathbf{C} \end{bmatrix} 
\begin{bmatrix} \mathbf{u}^{i-1} \\ \mathbf{p}^{i-1} \end{bmatrix} 
\end{align}

So on the right hand side, we have the standard residual, while the left hand side is an inexact linearization, with the inexact Jacobian given by a block triangular matrix.

## Task 1 - Understanding the fixed-strain split
Understand the following implementation fixed-strain splitting scheme. The fixed-strain solver has a similar structure to the monolithic solver. Remember, we only need to change the definition of the Jacobian (or more explicitly, how to solve the block triangular system). Tip 1: Search for "NEW". Tip 2: If you want to stop the solver, press on the "Stop" symbol on the top panel.

In [7]:
# Reset the model
model = FootingProblem(model_params)
model.prepare_simulation()
first_call = True
time_step = 0
num_dofs = model.num_dofs()
increment = np.zeros(num_dofs)
residual = np.zeros(num_dofs)

while model.time < model.end_time:
    # Time management
    time_step += 1
    model.time += model.time_step

    print(f"+=====================================+\nTime step {time_step}:") 

    # Newton loop
    model.before_newton_loop()
    while model.convergence_status == False:

        # Empty method, but include for completeness
        model.before_newton_iteration()

        ###########################################################
        
        # NEW: Fixed strain approximation
        
        # Set up the Jacobians of the two subproblems and factorize
        # them - only in the first iteration.
        if first_call:
            A, _ = model.assemble_linear_mechanics_system()
            C, _ = model.assemble_linear_flow_system()
            A_splu = sps.linalg.splu(A)
            C_splu = sps.linalg.splu(C)
            first_call = False
        
        # Require global dof number to address u and p separately
        u_dofs = model.displacement_dofs()
        p_dofs = model.pressure_dofs()

        # 1. Step: Solve the flow problem for given displacement
        _, residual[p_dofs] = model.assemble_linear_flow_system()
        increment[p_dofs] = C_splu.solve(residual[p_dofs])
        model.distribute_solution(increment, variables=["p"])
        
        # 2. Step: Solve the mechanics problem for updated pressre
        _, residual[u_dofs] = model.assemble_linear_mechanics_system()
        increment[u_dofs] = A_splu.solve(residual[u_dofs])
        model.distribute_solution(increment, variables=["u"])
        
        ###########################################################
        
        # Manage the iteration (accumulate to determine the solution),
        # and monitor convergence. Stop, if residual is below tolerance
        is_converged = model.after_newton_iteration(increment, residual, tol=1e-6)
        
        # Export solution
        if is_converged:
            model.after_newton_convergence()
            model.export(time_step)

    print(f"+=====================================+\n")

+=====================================+
Time step 1:


/home/jakub/anaconda3/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


Iter. 1 | Residual 79057.50171297793
Iter. 2 | Residual 10671.223938131723
Iter. 3 | Residual 1830.2795276386619
Iter. 4 | Residual 321.66348612270866
Iter. 5 | Residual 56.91695963833022
Iter. 6 | Residual 10.097161212681986
Iter. 7 | Residual 1.7934462604921455
Iter. 8 | Residual 0.31878657630652374
Iter. 9 | Residual 0.056696077809706494
Iter. 10 | Residual 0.01008814045881258
Iter. 11 | Residual 0.0017957842184834624
Iter. 12 | Residual 0.0003197921309873358
Iter. 13 | Residual 5.696916966309765e-05
Iter. 14 | Residual 1.01522054180851e-05
Iter. 15 | Residual 1.8097461760261314e-06
Iter. 16 | Residual 3.226812440488982e-07
+=====================================+

+=====================================+
Time step 2:
Iter. 1 | Residual 165.9146700718482
Iter. 2 | Residual 7.774657707021434
Iter. 3 | Residual 0.6348864867378158
Iter. 4 | Residual 0.07174177824543254
Iter. 5 | Residual 0.009823137892756966
Iter. 6 | Residual 0.0014881078532102325
Iter. 7 | Residual 0.000237662570920354

## Task 2 - Correlate coupling strength with the conditional robustness of fixed-strain
Check the coupling strength $\tau = \frac{\alpha^2}{s_0 K_\mathrm{dr}}$ of Biot's equations for the given material parameters. Use the following pre-defined routine for this.

In [15]:
def coupling_strength(params) -> float:

    Kdr = params["lame_lambda"] + 2 * params["lame_mu"] / 2.
    alpha = params["alpha"]
    s0 = params["s0"]
    s0inv = 1. / s0 if s0 > 1e-17 else float('inf')
    return alpha**2 / Kdr * s0inv

tau = coupling_strength(model_params)
print(f"The coupling strength is {tau}.")

The coupling strength is 0.18181818181818182.


Go back to the problem parameters. Choose them such they result in a coupling strength below, equal, and greater than 1. Check again the performance of the solver. For instance, multiply $\texttt{s0}$ with 0.17, 0.19, and 2.

## Fixed-stress split - A stabilized Block-Gauss Seidel solver
We have learned that for problems with saddle point structure, we require a good approximation of the Schur complement, in order for block-triangular approximations of the Jacobian to result in robust solvers. The exact Schur complement $\mathbf{S} = \mathbf{B}\mathbf{A}^{-1}\mathbf{B}$ for the Biot equations is spectrally equivalent with the pressure mass matrix $\mathbf{M}$. Thus, we can stabilize $\mathbf{C}$ in the fixed-strain split by a pressure mass matrix, scaled by $\frac{\alpha^2}{K_\mathrm{dr}}$ multiplied with a tuning factor $L$, resulting in $\hat{\mathbf{S}} = L\, \frac{\alpha^2}{K_\mathrm{dr}}\, \mathbf{M}$.

Iteration $i$ of the  stabilized Block-Gauss Seidel reads then: Given $\mathbf{u}^{i-1}$ and $p^{i-1}$, find $\mathbf{u}^i := \mathbf{u}^{i-1} + \Delta \mathbf{u}^i$ and $p^i := p^{i-1} + \Delta p^{i}$ with the increments satisfying

\begin{align}
\begin{bmatrix} \mathbf{A} & -\mathbf{B}^\top \\ \mathbf{0} & \mathbf{C} + \hat{\mathbf{S}}  \end{bmatrix} 
\begin{bmatrix} \Delta \mathbf{u}^i \\ \Delta \mathbf{p}^i \end{bmatrix} 
= \begin{bmatrix} \mathbf{f} \\ \mathbf{g} \end{bmatrix} - \begin{bmatrix} \mathbf{A} & -\mathbf{B}^\top \\ \mathbf{B} & \mathbf{C} \end{bmatrix} 
\begin{bmatrix} \mathbf{u}^{i-1} \\ \mathbf{p}^{i-1} \end{bmatrix} 
\end{align}



## Task 3 - From fixed-strain to fixed-stress

We start with a copy of the fixed-strain split from above. Check out the section with "NEW". Almost all ingredients for the fixed-stress split are provided. Modify the code and build your fixed-stress split.

$\textit{Advanced - can be ignored at the first read:}$
We define Anderson acceleration to potentially accelerate the convergence. To recall, it works similar to GMRES and reuses previous iterations when updating the next iteration in the context of iterative solvers. It works in a postprocessor fashion and is activated after each iteration of an iterative solver. By choosing $\texttt{aa_depth}$ one can define how many previous iterates are used. The choice $\texttt{aa_depth}=0$ deactives Anderson acceleration. This will be subject of a later task.

In [26]:
# Update some of the parameters
model_params["aa_depth"] = 5
model_params["mesh_resolution"] = 80
model_params["fixed_stress_L"] = 0
model_params["s0"] = 0.5e-8

# Reset the model
model = FootingProblem(model_params)
model.prepare_simulation()
first_call = True
time_step = 0
num_dofs = model.num_dofs()
increment = np.zeros(num_dofs)
residual = np.zeros(num_dofs)

# Anderson acceleration
from porepy.numerics.solvers.andersonacceleration import AndersonAcceleration
depth = model_params["aa_depth"]
aa = AndersonAcceleration(num_dofs, depth)

while model.time < model.end_time:
    # Time management
    time_step += 1
    model.time += model.time_step

    print(f"+=====================================+\nTime step {time_step}:") 

    # Reset Anderson acceleration
    aa.reset()

    # Newton loop
    model.before_newton_loop()
    while model.convergence_status == False:

        # Empty method, but include for completeness
        model.before_newton_iteration()

        ###########################################################
        
        # Splitting:
        
        # Set up the Jacobians of the two subproblems and factorize
        # them - only in the first iteration.
        if first_call:
            A, _ = model.assemble_linear_mechanics_system()
            C, _ = model.assemble_linear_flow_system()
            
            # NEW - TODO setup the stabilization!
            #mass_mat = model.pressure_mass_matrix()
            #alpha = model_params["alpha"]
            # TODO: Kdr = ... # This has been defined at other places in this notebook.
            #L = model_params["fixed_stress_L"]
            # TODO: fixed-stress stabilization? S = ...
            # TODO: How is S incorporated in the rest of the splitting scheme? 

            A_splu = sps.linalg.splu(A)
            C_splu = sps.linalg.splu(C + S)
            first_call = False
        
        # Require global dof number to address u and p separately
        u_dofs = model.displacement_dofs()
        p_dofs = model.pressure_dofs()

        # 1. Step: Solve the flow problem for given displacement
        _, residual[p_dofs] = model.assemble_linear_flow_system()
        increment[p_dofs] = C_splu.solve(residual[p_dofs])
        model.distribute_solution(increment, variables=["p"])
        
        # 2. Step: Solve the mechanics problem for updated pressre
        _, residual[u_dofs] = model.assemble_linear_mechanics_system()
        increment[u_dofs] = A_splu.solve(residual[u_dofs])
        model.distribute_solution(increment, variables=["u"])
        
        ###########################################################

        # Anderson acceleration (details can be ignored)
        if model.nonlinear_iteration() > 0:
            gk = model.dof_manager.assemble_variable(from_iterate=True)
            fk = increment
            improved_approximation = aa.apply(gk, fk, model.nonlinear_iteration()-1)
            model.distribute_solution(improved_approximation, overwrite=True)

        ###########################################################
        
        # Manage the iteration (accumulate to determine the solution),
        # and monitor convergence. Stop, if residual is below tolerance
        is_converged = model.after_newton_iteration(increment, residual, tol=1e-6)
        
        # Export solution
        if is_converged:
            model.after_newton_convergence()
            model.export(time_step)

    print(f"+=====================================+\n")

+=====================================+
Time step 1:


/home/jakub/anaconda3/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


Iter. 1 | Residual 79057.50171297793
Iter. 2 | Residual 91989.13750813385
Iter. 3 | Residual 151233.98743769364
Iter. 4 | Residual 9656.22148133375
Iter. 5 | Residual 1854.5908042316144
Iter. 6 | Residual 328.012733309489
Iter. 7 | Residual 97.43422108354449
Iter. 8 | Residual 29.3442607849402
Iter. 9 | Residual 9.572247591645738
Iter. 10 | Residual 2.288205148970342
Iter. 11 | Residual 0.48915823975051154
Iter. 12 | Residual 0.21622129919531233
Iter. 13 | Residual 0.06419306419156871
Iter. 14 | Residual 0.021460372778235735
Iter. 15 | Residual 0.008026052872567682
Iter. 16 | Residual 0.0028095138439083443
Iter. 17 | Residual 0.001296738432632207
Iter. 18 | Residual 0.0004354607431580531
Iter. 19 | Residual 0.00010795820432027183
Iter. 20 | Residual 5.2656919379144516e-05
Iter. 21 | Residual 2.0615319486814786e-05
Iter. 22 | Residual 8.067855590277947e-06
Iter. 23 | Residual 3.4714605710474884e-06
Iter. 24 | Residual 1.1434017494294248e-06
Iter. 25 | Residual 5.516918746494003e-07
+===

## Task 2 again - Unconditional robustness of the fixed-stress split
Revisit Task 2, but now for the fixed-stress split.

## Task 4 - Mesh independent convergence
Keep all material parameters fixed, and investigate how the number of iteration evolves when changing the mesh size via $\texttt{model_params["mesh_resolution"]}$. Choose for instance resolutions of 40 and 160. Verify that the convergence behavior of the fixed-stress split is independent of the mesh size.

## Task 5 - Accelerate convergence by turning on Anderson acceleration.
Increase the depth of the Anderson acceleration via $\texttt{model_params["aa_depth"]}$. Typical values for the depath are 1, 5 or 10. Also try $L=0$ equivalent with the fixed-strain split. Does the scheme converge even when $\tau>1$? How far can you push it?